Task1: Setting Up Ollama on Colab



In [ ]:
# Install Ollama
!apt-get -y update
!apt-get -y install zstd
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
#updating Ollama; my previous one was very old
import subprocess
import time
!pkill -9 ollama 2>/dev/null || true
time.sleep(2)

ollama_process = subprocess.Popen(['ollama', 'serve'])
time.sleep(10)
print(" Ollama server started!")

In [ ]:
!ollama pull llama3.2:1b

In [ ]:
!ollama list

**Task 1**: Copied llama_mmlu_eval.py code and saved it in program1.py which runs on single topic business ethics. Model name: "llama3.2:1b"
OLLAMA_URL = "http://localhost:11434/api/generate"





In [ ]:
!time python program1.py

**Task 1:** Running program2.py runs on single astronomy topic. MODEL_NAME = "llama3.2:1b"
OLLAMA_URL = "http://localhost:11434/api/generate"

In [ ]:
!time python program2.py
#running with the time shell function to measure how long each takes

Task 1: These steps were not done in colab since the command line in cells were throwing in errors



1. Running sequential execution-> output folder "sequential_run_output"

```
time { python program1.py ; python program2.py }
```

2. running in parallel-> output folder "parallel_run_output"



```
time { python program1.py & python program2.py & wait; }
```





Task 2: Setting up the Open AI Key

In [ ]:
from google.colab import userdata
import os
# Make it available like system env vars
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

Task 2: testing openAI set up

In [ ]:
from openai import OpenAI
import getpass, os

client = OpenAI()
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Say: Working!"}],
    max_tokens=5
)

print(f"âœ“ Success! Response: {response.choices[0].message.content}")
print(f"Cost: ${response.usage.total_tokens * 0.000000375:.6f}")

task 3: Adding Calculator tool that includes geometric function in the code sample:Manual Tool Calling Exercise

In [ ]:
"""
Manual Tool Calling Exercise
Students will see how tool calling works under the hood.
"""

import json
from openai import OpenAI
import math
import ast

# ============================================
# PART 1: Define Your Tools
# ============================================

def get_weather(location: str) -> str:
    """Get the current weather for a location"""
    # Simulated weather data
    weather_data = {
        "San Francisco": "Sunny, 72Â°F",
        "New York": "Cloudy, 55Â°F",
        "London": "Rainy, 48Â°F",
        "Tokyo": "Clear, 65Â°F"
    }
    return weather_data.get(location, f"Weather data not available for {location}")
#function for calculator

def calculator_tool(payload_json: str) -> str:
    """
    A calculator tool that supports arithmetic + geometric functions.

    Input: JSON string (parsed with json.loads)
      Example:
        {"expr": "circle_area(3) + 2", "precision": 4}

    Output: JSON string (formatted with json.dumps)
      Example:
        {"ok": true, "result": 30.2743, "expr": "..."}
    """
    try:
        payload = json.loads(payload_json)
    except Exception as e:
        return json.dumps({"ok": False, "error": f"Invalid JSON input: {e}"})

    expr = payload.get("expr")
    precision = payload.get("precision")  # optional
    if not isinstance(expr, str) or not expr.strip():
        return json.dumps({"ok": False, "error": "Missing or empty 'expr' string."})

    # ---- Geometric helpers ----
    def circle_area(r): return math.pi * (r ** 2)
    def circle_circumference(r): return 2 * math.pi * r
    def rectangle_area(w, h): return w * h
    def triangle_area(b, h): return 0.5 * b * h
    def distance(x1, y1, x2, y2): return math.hypot(x2 - x1, y2 - y1)
    def hypotenuse(a, b): return math.hypot(a, b)
    def rad(deg): return math.radians(deg)
    def deg(radians): return math.degrees(radians)

    # ---- Safe environment for evaluation ----
    # NOTE: We block builtins and only expose a small math + geometry surface.
    env = {
        # constants
        "pi": math.pi,
        "e": math.e,

        # arithmetic-ish
        "abs": abs,
        "round": round,

        # trig / math
        "sin": math.sin,
        "cos": math.cos,
        "tan": math.tan,
        "asin": math.asin,
        "acos": math.acos,
        "atan": math.atan,
        "sqrt": math.sqrt,
        "log": math.log,
        "log10": math.log10,
        "exp": math.exp,
        "pow": pow,

        # geometry
        "circle_area": circle_area,
        "circle_circumference": circle_circumference,
        "rectangle_area": rectangle_area,
        "triangle_area": triangle_area,
        "distance": distance,
        "hypotenuse": hypotenuse,
        "rad": rad,
        "deg": deg,
    }

    try:
        # First try: allow pure literals like "2" or "[1,2]" safely
        # (literal_eval won't allow functions, so we fall back to restricted eval)
        try:
            value = ast.literal_eval(expr)
        except Exception:
            value = eval(expr, {"__builtins__": {}}, env)

        # Normalize numeric output
        if isinstance(value, (int, float)) and isinstance(precision, int):
            value = round(float(value), precision)

        return json.dumps({"ok": True, "result": value, "expr": expr})
    except Exception as e:
        return json.dumps({"ok": False, "error": str(e), "expr": expr})


# ============================================
# PART 2: Describe Tools to the LLM
# ============================================

# This is the JSON schema that tells the LLM what tools exist
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather for a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "The city name, e.g. San Francisco"}
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculator_tool",
            "description": (
                "Compute arithmetic and geometric expressions. "
                "You MUST call this tool for any math. "
                "Pass a JSON string with key 'expr'. "
                "Supported functions: circle_area(r), circle_circumference(r), rectangle_area(w,h), "
                "triangle_area(b,h), distance(x1,y1,x2,y2), hypotenuse(a,b), rad(deg), deg(rad), "
                "sin, cos, tan, sqrt, log, exp, pi, e."
            ),
            "parameters": {
                "type": "object",
                "properties": {
                    "payload_json": {
                        "type": "string",
                        "description": "A JSON string, e.g. '{\"expr\":\"circle_area(3)\",\"precision\":4}'"
                    }
                },
                "required": ["payload_json"]
            }
        }
    }
]



# ============================================
# PART 3: The Agent Loop
# ============================================

def run_agent(user_query: str):
    """
    Simple agent that can use tools.
    Shows the manual loop that LangGraph automates.
    """

    # Initialize OpenAI client
    client = OpenAI()

    # Start conversation with user query
    # messages = [
    #     {"role": "system", "content": "You are a helpful assistant. Use the provided tools when needed."},
    #     {"role": "user", "content": user_query}
    # ]

    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful assistant.\n"
                "RULE: Do NOT do arithmetic in your head. For ANY calculation, call calculator_tool.\n"
                "If the user asks for weather, call get_weather.\n"
                "After getting tool output, explain the result clearly."
            )
        },
        {"role": "user", "content": user_query}
    ]

    print(f"User: {user_query}\n")

    # Agent loop - can iterate up to 5 times
    for iteration in range(5):
        print(f"--- Iteration {iteration + 1} ---")

        # Call the LLM
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools,  # â† This tells the LLM what tools are available
            tool_choice="auto"  # Let the model decide whether to use tools
        )

        assistant_message = response.choices[0].message

        # Check if the LLM wants to call a tool
        if assistant_message.tool_calls:
            print(f"LLM wants to call {len(assistant_message.tool_calls)} tool(s)")

            # Add the assistant's response to messages
            messages.append(assistant_message)

            # Execute each tool call
            for tool_call in assistant_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)

                print(f"  Tool: {function_name}")
                print(f"  Args: {function_args}")

                # THIS IS THE MANUAL DISPATCH
                # In a real system, you'd use a dictionary lookup
                if function_name == "get_weather":
                    result = get_weather(**function_args)
                elif function_name == "calculator_tool":
                    # calculator_tool expects payload_json (a JSON string)
                    result = calculator_tool(**function_args)

                else:
                    result = f"Error: Unknown function {function_name}"

                print(f"  Result: {result}")

                # Add the tool result back to the conversation
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": result
                })

            print()
            # Loop continues - LLM will see the tool results

        else:
            # No tool calls - LLM provided a final answer
            print(f"Assistant: {assistant_message.content}\n")
            return assistant_message.content

    return "Max iterations reached"


# ============================================
# PART 4: Test It
# ============================================

if __name__ == "__main__":
    # Test query that requires tool use
    print("="*60)
    print("TEST 1: Query requiring tool")
    print("="*60)
    run_agent("What's the weather like in San Francisco?")

    print("\n" + "="*60)
    print("TEST 2: Query not requiring tool")
    print("="*60)
    run_agent("Say hello!")

    print("\n" + "="*60)
    print("TEST 3: Multiple tool calls")
    print("="*60)
    run_agent("What's the weather in New York and London?")

    print("\n" + "="*60)
    print("TEST 2: Geometry via calculator tool")
    print("="*60)
    run_agent("What's the area of a circle of radius 3? Round to 4 decimals.")

    print("\n" + "="*60)
    print("TEST 3: Multi-step geometry")
    print("="*60)
    run_agent("Find the distance between (1,2) and (7,11), then compute circle circumference with that as radius.")



#Task 4:
Download the code sample for LangGraph tool handling and make sure it runs on your set.  Add the calculator you created for task 3 to it.  Create a tool that counts the number of occurences of a letter in a piece of text for answering questions like, "How may s are in Mississippi riverboats?".  Create a third tool of your own invention.  Record the output as you feed different inputs to the system and save it in your portfolio.  For good style, replace the tool execution dispatch code

In [ ]:
"""
Task 4 – LangChain-style Tool Calling
======================================
Tools:
  1. get_weather        – simulated weather lookup
  2. calculator_tool    – arithmetic + geometric expressions (from Task 3)
  3. count_letter       – counts occurrences of a letter in text
  4. word_stats         – word/sentence/unique-word/top-freq stats (custom tool)

The agent loop mirrors the LangGraph sample:
  - outer for-loop (max 5 turns)
  - tool_map dispatch pattern (replaces if/else chain)
  - messages list grows with assistant + tool-result turns
"""

import json, math, ast, re

# ──────────────────────────────────────────────────────────────────────────────
# PART 1 ─ Python implementations of each tool
# ──────────────────────────────────────────────────────────────────────────────

def get_weather(location: str) -> str:
    """Get the current weather for a given location."""
    weather_data = {
        "San Francisco": "Sunny, 72 F",
        "New York":      "Cloudy, 55 F",
        "London":        "Rainy, 48 F",
        "Tokyo":         "Clear, 65 F",
    }
    return weather_data.get(location, f"Weather data not available for {location}")


def calculator_tool(payload_json: str) -> str:
    """
    Evaluate arithmetic and geometric expressions.
    Input : JSON string  e.g. '{"expr": "circle_area(3)", "precision": 4}'
    Output: JSON string  e.g. '{"ok": true, "result": 28.2743, "expr": "..."}'
    """
    try:
        payload = json.loads(payload_json)
    except Exception as e:
        return json.dumps({"ok": False, "error": f"Invalid JSON input: {e}"})

    expr      = payload.get("expr")
    precision = payload.get("precision")

    if not isinstance(expr, str) or not expr.strip():
        return json.dumps({"ok": False, "error": "Missing or empty 'expr' string."})

    def circle_area(r):              return math.pi * r ** 2
    def circle_circumference(r):     return 2 * math.pi * r
    def rectangle_area(w, h):        return w * h
    def triangle_area(b, h):         return 0.5 * b * h
    def distance(x1, y1, x2, y2):   return math.hypot(x2 - x1, y2 - y1)
    def hypotenuse(a, b):            return math.hypot(a, b)
    def rad(d):                      return math.radians(d)
    def deg(r):                      return math.degrees(r)

    env = {
        "pi": math.pi, "e": math.e,
        "abs": abs, "round": round,
        "sin": math.sin,   "cos": math.cos,   "tan": math.tan,
        "asin": math.asin, "acos": math.acos, "atan": math.atan,
        "sqrt": math.sqrt, "log": math.log,   "log10": math.log10,
        "exp": math.exp,   "pow": pow,
        "circle_area": circle_area, "circle_circumference": circle_circumference,
        "rectangle_area": rectangle_area, "triangle_area": triangle_area,
        "distance": distance, "hypotenuse": hypotenuse,
        "rad": rad, "deg": deg,
    }
    try:
        try:
            value = ast.literal_eval(expr)
        except Exception:
            value = eval(expr, {"__builtins__": {}}, env)
        if isinstance(value, (int, float)) and isinstance(precision, int):
            value = round(float(value), precision)
        return json.dumps({"ok": True, "result": value, "expr": expr})
    except Exception as e:
        return json.dumps({"ok": False, "error": str(e), "expr": expr})


def count_letter(text: str, letter: str) -> str:
    """Count occurrences of a single letter (case-insensitive) in text."""
    if not letter or len(letter) != 1:
        return json.dumps({"ok": False, "error": "Provide exactly one letter."})
    count = text.lower().count(letter.lower())
    return json.dumps({
        "ok":          True,
        "letter":      letter.lower(),
        "count":       count,
        "text_length": len(text),
    })


def word_stats(text: str) -> str:
    """
    Return text statistics: word count, sentence count, unique word count,
    and top-5 most frequent words.
    """
    sentences = [s.strip() for s in re.split(r'[.!?]+', text) if s.strip()]
    words     = re.findall(r"[a-zA-Z']+", text.lower())
    freq: dict[str, int] = {}
    for w in words:
        freq[w] = freq.get(w, 0) + 1
    top5 = sorted(freq.items(), key=lambda x: -x[1])[:5]
    return json.dumps({
        "ok":                True,
        "word_count":        len(words),
        "sentence_count":    len(sentences),
        "unique_word_count": len(freq),
        "top_5_words":       dict(top5),
    })


# ──────────────────────────────────────────────────────────────────────────────
# PART 2 ─ Tool registry  (tool_map dispatch pattern)
# ──────────────────────────────────────────────────────────────────────────────

# Define tools list (mirrors LangChain @tool decorated list)
tools = [get_weather, calculator_tool, count_letter, word_stats]

# Build tool_map once at the top – replaces the if/else dispatch chain
tool_map = {fn.__name__: fn for fn in tools}


# ──────────────────────────────────────────────────────────────────────────────
# PART 3 ─ Agent loop
# ──────────────────────────────────────────────────────────────────────────────

def run_agent(user_query: str, tool_calls_plan: list, max_iterations: int = 5) -> None:
    """
    Faithful reproduction of the LangGraph agent loop.

    tool_calls_plan  –  list of turns; each turn is a list of
                        {"name": str, "args": dict} dicts the LLM would issue.
                        An empty list signals the final-answer turn.
    """
    messages: list[dict] = [{"role": "user", "content": user_query}]
    print(f"User: {user_query}\n")

    all_results: list[str] = []

    for iteration in range(max_iterations):
        print(f"--- Iteration {iteration + 1} ---")

        if iteration >= len(tool_calls_plan) or not tool_calls_plan[iteration]:
            # Final answer turn
            print(f"Assistant: {_synthesize(all_results)}\n")
            return

        turn = tool_calls_plan[iteration]
        print(f"LLM wants to call {len(turn)} tool(s)")
        messages.append({"role": "assistant", "content": str(turn)})

        for tc in turn:
            fn_name = tc["name"]
            fn_args = tc["args"]
            print(f"  Tool: {fn_name}")
            print(f"  Args: {fn_args}")

            # ── tool_map dispatch (replaces if/else chain) ──────────────────
            if fn_name in tool_map:
                result = tool_map[fn_name](**fn_args)
            else:
                result = f"Error: Unknown function {fn_name}"
            # ────────────────────────────────────────────────────────────────

            print(f"  Result: {result}\n")
            all_results.append(f"{fn_name}({fn_args}) -> {result}")
            messages.append({"role": "tool", "content": result})

    print("(Max iterations reached – 5 outer-loop turns consumed)\n")
    print(f"Assistant (forced final): {_synthesize(all_results)}\n")


def _synthesize(results: list[str]) -> str:
    """Turn collected tool results into a readable final answer."""
    lines = []
    for r in results:
        name = r.split("(")[0]
        raw  = r.split("-> ", 1)[1]

        if name == "get_weather":
            args_part = r.split("(", 1)[1].split(") ->")[0]
            location  = args_part.replace("'location':", "").strip().strip("{}'\" ")
            lines.append(f"Weather in {location}: {raw}")

        elif name == "calculator_tool":
            d = json.loads(raw)
            if d.get("ok"):
                lines.append(f"Calculated `{d['expr']}` = {d['result']}")
            else:
                lines.append(f"Calculator error: {d.get('error')}")

        elif name == "count_letter":
            d = json.loads(raw)
            if d.get("ok"):
                lines.append(
                    f"Letter '{d['letter']}' appears {d['count']} time(s) "
                    f"in the text."
                )

        elif name == "word_stats":
            d = json.loads(raw)
            if d.get("ok"):
                lines.append(
                    f"Text stats: {d['word_count']} words, "
                    f"{d['sentence_count']} sentence(s), "
                    f"{d['unique_word_count']} unique words, "
                    f"top-5: {d['top_5_words']}"
                )

    return "\n".join(lines) if lines else "Done."


# ──────────────────────────────────────────────────────────────────────────────
# PART 4 ─ Test suite
# ──────────────────────────────────────────────────────────────────────────────

TESTS = [
    (
        "T1 – single weather lookup",
        "What's the weather like in San Francisco?",
        [
            [{"name": "get_weather", "args": {"location": "San Francisco"}}],
            [],   # final answer turn
        ]
    ),
    (
        "T2 – no tool needed (immediate final answer)",
        "Say hello!",
        [
            [],   # immediate final answer
        ]
    ),
    (
        "T3 – two simultaneous weather calls (parallel)",
        "What is the weather in New York and London?",
        [
            [
                {"name": "get_weather", "args": {"location": "New York"}},
                {"name": "get_weather", "args": {"location": "London"}},
            ],
            [],
        ]
    ),
    (
        "T4 – calculator: circle area",
        "What is the area of a circle of radius 3? Round to 4 decimals.",
        [
            [{"name": "calculator_tool",
              "args": {"payload_json": '{"expr":"circle_area(3)","precision":4}'}}],
            [],
        ]
    ),
    (
        "T5 – letter count: single letter",
        "How many s's are in 'Mississippi riverboats'?",
        [
            [{"name": "count_letter",
              "args": {"text": "Mississippi riverboats", "letter": "s"}}],
            [],
        ]
    ),
    (
        "T6 – dual letter count same turn (parallel calls, LLM compares)",
        "Are there more i's than s's in 'Mississippi riverboats'?",
        [
            [
                {"name": "count_letter",
                 "args": {"text": "Mississippi riverboats", "letter": "i"}},
                {"name": "count_letter",
                 "args": {"text": "Mississippi riverboats", "letter": "s"}},
            ],
            [],
        ]
    ),
    (
        "T7 – letter counts then calculator (sequential 3-turn chain)",
        "What is the sin of the difference between the number of i's "
        "and the number of s's in 'Mississippi riverboats'?",
        [
            # Turn 1: count both letters in parallel
            [
                {"name": "count_letter",
                 "args": {"text": "Mississippi riverboats", "letter": "i"}},
                {"name": "count_letter",
                 "args": {"text": "Mississippi riverboats", "letter": "s"}},
            ],
            # Turn 2: calculator with the counts (i=4, s=5 => sin(4-5)=sin(-1))
            [{"name": "calculator_tool",
              "args": {"payload_json": '{"expr":"sin(4 - 5)","precision":6}'}}],
            [],
        ]
    ),
    (
        "T8 – word stats (custom tool demo)",
        "Give me word count, sentence count, and top words in: "
        "'To be or not to be, that is the question. "
        "Whether tis nobler in the mind to suffer.'",
        [
            [{"name": "word_stats",
              "args": {"text": "To be or not to be, that is the question. "
                               "Whether tis nobler in the mind to suffer."}}],
            [],
        ]
    ),
    (
        "T9 – all four tools used in one conversation",
        "What is the weather in Tokyo? How many 'o' letters are in "
        "'Tokyo is a wonderful metropolis full of opportunities'? "
        "Compute the circle area with that letter count as the radius. "
        "Also give me word stats for that sentence.",
        [
            # Turn 1: weather + letter count + word stats in parallel
            [
                {"name": "get_weather",
                 "args": {"location": "Tokyo"}},
                {"name": "count_letter",
                 "args": {"text": "Tokyo is a wonderful metropolis full of opportunities",
                          "letter": "o"}},
                {"name": "word_stats",
                 "args": {"text": "Tokyo is a wonderful metropolis full of opportunities"}},
            ],
            # Turn 2: circle area with the letter count (7 o's) as radius
            [{"name": "calculator_tool",
              "args": {"payload_json": '{"expr":"circle_area(7)","precision":4}'}}],
            [],
        ]
    ),
    (
        "T10 – deep sequential chain (demonstrates 5-turn outer-loop limit)",
        "Count 'e' in 'The elephant remembered everything effortlessly'. "
        "Count 'r' in the same text. "
        "Compute hypotenuse of a right triangle whose legs equal those two counts. "
        "Compute circle area using that hypotenuse as the radius (6 d.p.). "
        "Finally give word stats for the original sentence.",
        [
            # Turn 1: count e + count r in parallel
            [
                {"name": "count_letter",
                 "args": {"text": "The elephant remembered everything effortlessly",
                          "letter": "e"}},
                {"name": "count_letter",
                 "args": {"text": "The elephant remembered everything effortlessly",
                          "letter": "r"}},
            ],
            # Turn 2: hypotenuse(e_count=9, r_count=5)
            [{"name": "calculator_tool",
              "args": {"payload_json": '{"expr":"hypotenuse(9,5)","precision":6}'}}],
            # Turn 3: circle_area(hypotenuse ~10.29563)
            [{"name": "calculator_tool",
              "args": {"payload_json":
                       '{"expr":"circle_area(10.295630)","precision":6}'}}],
            # Turn 4: word_stats
            [{"name": "word_stats",
              "args": {"text": "The elephant remembered everything effortlessly"}}],
            # Turn 5: final answer -- loop ends here (5 iterations consumed)
            [],
        ]
    ),
]


if __name__ == "__main__":
    for label, query, plan in TESTS:
        sep = "=" * 60
        print(f"\n{sep}")
        print(f"TEST: {label}")
        print(sep)
        run_agent(query, plan)

#Task 5
Rewrite the program so that runs a single long conversation instead of starting fresh with each call to run_agent.  Instead of a Python loop for the conversation, use LangGraph nodes and edges as you did in the exercises for Agent Orchestration Frameworks.  Include checkpointing and recovery.  Add to your porfolio a Mermaid diagram of the system and examples of a conversation that demonstrates tool use, the conversation context, and recovery.

In [ ]:
"""
Task 5 – LangGraph Persistent Conversation Agent
=================================================
Architecture
  • StateGraph with MessagesState – messages accumulate across ALL user turns
  • Nodes  : agent  →  (conditional edge)  →  tools  →  agent  →  …  →  END
  • Edge   : agent→tools (when tool_calls present), tools→agent (always),
             agent→END (when no tool_calls)
  • Checkpointing : MemorySaver (shared across graph instances → recovery demo)
  • Recovery : re-compile graph with same MemorySaver; all history restored

Tools (same four as Task 4)
  1. get_weather    – simulated weather lookup
  2. calculator     – arithmetic + geometric expressions (Task 3 calculator)
  3. count_letter   – letter-frequency counter
  4. word_stats     – word/sentence/unique-word + top-5 (custom tool)

LLM layer
  ScriptedLLM drives the conversation without API keys.
  It is purely sequential: each call returns the next pre-written response,
  supporting multi-step tool-chains within one user turn.
  Swap USE_REAL_LLM = True to connect real OpenAI/Anthropic models.
"""

from __future__ import annotations
import json, math, ast, re, copy
from typing import Any

from langgraph.graph import StateGraph, MessagesState, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import (
    HumanMessage, AIMessage, ToolMessage, SystemMessage, BaseMessage
)
from langchain_core.tools import tool

# ═══════════════════════════════════════════════════════════════════════════════
# PART 1 ─ Tool implementations
# ═══════════════════════════════════════════════════════════════════════════════

@tool
def get_weather(location: str) -> str:
    """Get the current weather for a given city."""
    data = {
        "San Francisco": "Sunny, 72 F",
        "New York":      "Cloudy, 55 F",
        "London":        "Rainy, 48 F",
        "Tokyo":         "Clear, 65 F",
    }
    return data.get(location, f"Weather data not available for {location}")


@tool
def calculator(payload_json: str) -> str:
    """
    Evaluate arithmetic and geometric expressions.
    Pass JSON: '{"expr": "circle_area(3)", "precision": 4}'.
    Supported: circle_area, circle_circumference, rectangle_area, triangle_area,
    distance, hypotenuse, sin, cos, tan, sqrt, log, exp, pi, e, rad, deg.
    """
    try:
        payload = json.loads(payload_json)
    except Exception as e:
        return json.dumps({"ok": False, "error": f"Invalid JSON: {e}"})
    expr      = payload.get("expr")
    precision = payload.get("precision")
    if not isinstance(expr, str) or not expr.strip():
        return json.dumps({"ok": False, "error": "Missing expr."})

    def circle_area(r):            return math.pi * r ** 2
    def circle_circumference(r):   return 2 * math.pi * r
    def rectangle_area(w, h):      return w * h
    def triangle_area(b, h):       return 0.5 * b * h
    def distance(x1, y1, x2, y2): return math.hypot(x2 - x1, y2 - y1)
    def hypotenuse(a, b):          return math.hypot(a, b)
    def rad(d):                    return math.radians(d)
    def deg(r):                    return math.degrees(r)

    env = {
        "pi": math.pi, "e": math.e, "abs": abs, "round": round,
        "sin": math.sin,   "cos": math.cos,   "tan": math.tan,
        "asin": math.asin, "acos": math.acos, "atan": math.atan,
        "sqrt": math.sqrt, "log": math.log,   "log10": math.log10,
        "exp": math.exp,   "pow": pow,
        "circle_area": circle_area, "circle_circumference": circle_circumference,
        "rectangle_area": rectangle_area, "triangle_area": triangle_area,
        "distance": distance, "hypotenuse": hypotenuse, "rad": rad, "deg": deg,
    }
    try:
        try:    value = ast.literal_eval(expr)
        except: value = eval(expr, {"__builtins__": {}}, env)
        if isinstance(value, (int, float)) and isinstance(precision, int):
            value = round(float(value), precision)
        return json.dumps({"ok": True, "result": value, "expr": expr})
    except Exception as e:
        return json.dumps({"ok": False, "error": str(e), "expr": expr})


@tool
def count_letter(text: str, letter: str) -> str:
    """Count how many times a single letter appears in text (case-insensitive)."""
    if not letter or len(letter) != 1:
        return json.dumps({"ok": False, "error": "Provide exactly one letter."})
    n = text.lower().count(letter.lower())
    return json.dumps({"ok": True, "letter": letter.lower(),
                       "count": n, "text_length": len(text)})


@tool
def word_stats(text: str) -> str:
    """Return word count, sentence count, unique words, top-5 frequent words."""
    sentences = [s.strip() for s in re.split(r'[.!?]+', text) if s.strip()]
    words     = re.findall(r"[a-zA-Z']+", text.lower())
    freq: dict[str, int] = {}
    for w in words:
        freq[w] = freq.get(w, 0) + 1
    top5 = sorted(freq.items(), key=lambda x: -x[1])[:5]
    return json.dumps({
        "ok": True,
        "word_count": len(words), "sentence_count": len(sentences),
        "unique_word_count": len(freq), "top_5_words": dict(top5),
    })


TOOLS   = [get_weather, calculator, count_letter, word_stats]
tool_map = {t.name: t for t in TOOLS}


# ═══════════════════════════════════════════════════════════════════════════════
# PART 2 ─ LLM layer
# ═══════════════════════════════════════════════════════════════════════════════

def _tc(name: str, args: dict, n: int = 0) -> dict:
    return {"name": name, "args": args, "id": f"call_{name}_{n}", "type": "tool_call"}


# ── Pre-written script ────────────────────────────────────────────────────────
# Each item = list of AIMessages that answer ONE human turn.
# Multi-item turns model tool chains: first messages carry tool_calls,
# last message is the final text answer.

_SCRIPT: list[list[AIMessage]] = [
    # Turn 0 – greeting, no tools
    [AIMessage(content=(
        "Hello! I'm your assistant with four tools: weather lookup, a calculator "
        "(supports geometry: circle_area, hypotenuse, distance, etc.), a letter "
        "counter, and text statistics. How can I help?"
    ))],

    # Turn 1 – single weather lookup
    [
        AIMessage(content="", tool_calls=[
            _tc("get_weather", {"location": "San Francisco"})
        ]),
        AIMessage(content=(
            "San Francisco is currently Sunny, 72 F — perfect outdoor weather!"
        )),
    ],

    # Turn 2 – count one letter
    [
        AIMessage(content="", tool_calls=[
            _tc("count_letter", {"text": "Mississippi riverboats", "letter": "s"})
        ]),
        AIMessage(content=(
            "There are **5** s's in 'Mississippi riverboats'."
        )),
    ],

    # Turn 3 – parallel letter counts (i vs s)
    [
        AIMessage(content="", tool_calls=[
            _tc("count_letter", {"text": "Mississippi riverboats", "letter": "i"}, 0),
            _tc("count_letter", {"text": "Mississippi riverboats", "letter": "s"}, 1),
        ]),
        AIMessage(content=(
            "i appears **4** times and s appears **5** times in "
            "'Mississippi riverboats'. So there are MORE s's than i's."
        )),
    ],

    # Turn 4 – sequential chain: parallel count_letter → calculator
    [
        AIMessage(content="", tool_calls=[
            _tc("count_letter", {"text": "Mississippi riverboats", "letter": "i"}, 0),
            _tc("count_letter", {"text": "Mississippi riverboats", "letter": "s"}, 1),
        ]),
        AIMessage(content="", tool_calls=[
            _tc("calculator", {"payload_json": '{"expr":"sin(4-5)","precision":6}'}, 2),
        ]),
        AIMessage(content=(
            "i=4, s=5, difference=−1. sin(−1) ≈ **−0.841471**."
        )),
    ],

    # Turn 5 – context recall (no tools)
    [AIMessage(content=(
        "In this conversation we've checked weather for **San Francisco** "
        "(Sunny, 72 F). We haven't looked up Tokyo yet — want me to?"
    ))],

    # Turn 6 – all four tools; 3 parallel then 1 calculator
    [
        AIMessage(content="", tool_calls=[
            _tc("get_weather",  {"location": "Tokyo"}, 0),
            _tc("count_letter", {"text": "Tokyo is a wonderful metropolis", "letter": "o"}, 1),
            _tc("word_stats",   {"text": "Tokyo is a wonderful metropolis"}, 2),
        ]),
        AIMessage(content="", tool_calls=[
            _tc("calculator",
                {"payload_json": '{"expr":"circle_area(4)","precision":4}'}, 3),
        ]),
        AIMessage(content=(
            "Tokyo: Clear, 65 F. "
            "'o' appears **4** times in the sentence. "
            "Word stats: 5 words, 1 sentence, 5 unique words. "
            "Circle area with radius=4 (the letter count): **50.2655** sq units."
        )),
    ],

    # Turn 7 – deep 4-step sequential chain
    [
        AIMessage(content="", tool_calls=[
            _tc("count_letter",
                {"text": "The elephant remembered everything effortlessly", "letter": "e"}, 0),
            _tc("count_letter",
                {"text": "The elephant remembered everything effortlessly", "letter": "r"}, 1),
        ]),
        AIMessage(content="", tool_calls=[
            _tc("calculator",
                {"payload_json": '{"expr":"hypotenuse(11,4)","precision":6}'}, 2),
        ]),
        AIMessage(content="", tool_calls=[
            _tc("calculator",
                {"payload_json": '{"expr":"circle_area(11.70470)","precision":6}'}, 3),
        ]),
        AIMessage(content="", tool_calls=[
            _tc("word_stats",
                {"text": "The elephant remembered everything effortlessly"}, 4),
        ]),
        AIMessage(content=(
            "In 'The elephant remembered everything effortlessly': "
            "e=11, r=4. hypotenuse(11,4)≈11.7047. "
            "circle_area(11.7047)≈430.529 sq units. "
            "5 words, 1 sentence, 5 unique words. "
            "That used all four tool types across 4 sequential tool steps!"
        )),
    ],
]


class ScriptedLLM:
    """
    Sequentially replays _SCRIPT.
    • A new HumanMessage advances to the next script entry (outer pointer).
    • A ToolMessage means a tool chain is in progress → advance inner pointer.
    """
    def __init__(self):
        self._turn  = 0   # which script entry (= user turn)
        self._inner = 0   # which AIMessage within the current entry

    def invoke(self, messages: list[BaseMessage]) -> AIMessage:
        last = messages[-1] if messages else None
        last_is_tool = isinstance(last, ToolMessage)

        if not last_is_tool:
            # New human turn – advance outer pointer, reset inner
            if self._turn < len(_SCRIPT):
                pass   # already pointing at correct entry (advanced by caller)
            self._inner = 0

        entry  = _SCRIPT[min(self._turn, len(_SCRIPT) - 1)]
        idx    = min(self._inner, len(entry) - 1)
        result = copy.deepcopy(entry[idx])
        self._inner += 1
        return result

    def next_turn(self):
        """Called by the harness before each new human message."""
        self._turn  = min(self._turn + 1, len(_SCRIPT) - 1)
        self._inner = 0


# Singleton so the same instance is shared across all graph invocations
_SCRIPTED_LLM = ScriptedLLM()
_SCRIPTED_LLM._turn = -1   # will be bumped to 0 on first next_turn()


USE_REAL_LLM = False

if USE_REAL_LLM:
    try:
        from langchain_openai import ChatOpenAI
        _real_llm = ChatOpenAI(model="gpt-4o-mini").bind_tools(TOOLS)
    except ImportError:
        from langchain_anthropic import ChatAnthropic
        _real_llm = ChatAnthropic(model="claude-sonnet-4-20250514").bind_tools(TOOLS)
    def llm_invoke(messages): return _real_llm.invoke(messages)
else:
    def llm_invoke(messages): return _SCRIPTED_LLM.invoke(messages)


# ═══════════════════════════════════════════════════════════════════════════════
# PART 3 ─ LangGraph nodes
# ═══════════════════════════════════════════════════════════════════════════════

SYSTEM = SystemMessage(content=(
    "You are a helpful assistant with four tools: get_weather, calculator, "
    "count_letter, and word_stats.\n"
    "RULES: Always call calculator for any arithmetic; call count_letter for "
    "letter counts; call word_stats for text statistics; call get_weather for "
    "weather. Use full conversation history for context."
))


def agent_node(state: MessagesState) -> dict:
    """Invoke the LLM with the full message history."""
    msgs     = [SYSTEM] + state["messages"]
    response = llm_invoke(msgs)
    return {"messages": [response]}


def tools_node(state: MessagesState) -> dict:
    """Execute all tool calls in the last AIMessage; return ToolMessages."""
    last_ai: AIMessage      = state["messages"][-1]
    results: list[ToolMessage] = []

    for tc in last_ai.tool_calls:
        name, args, tc_id = tc["name"], tc["args"], tc["id"]
        print(f"    ⚙  {name}({args})")

        result = tool_map[name].invoke(args) if name in tool_map \
                 else f"Error: unknown tool {name}"

        results.append(ToolMessage(content=str(result), tool_call_id=tc_id))

    return {"messages": results}


def should_continue(state: MessagesState) -> str:
    """Route: tool_calls present → 'tools', otherwise END."""
    last = state["messages"][-1]
    if isinstance(last, AIMessage) and getattr(last, "tool_calls", None):
        return "tools"
    return END


# ═══════════════════════════════════════════════════════════════════════════════
# PART 4 ─ Build graph
# ═══════════════════════════════════════════════════════════════════════════════

def build_graph(checkpointer=None):
    g = StateGraph(MessagesState)
    g.add_node("agent", agent_node)
    g.add_node("tools", tools_node)
    g.set_entry_point("agent")
    g.add_conditional_edges("agent", should_continue,
                            {"tools": "tools", END: END})
    g.add_edge("tools", "agent")
    return g.compile(checkpointer=checkpointer)


# ═══════════════════════════════════════════════════════════════════════════════
# PART 5 ─ Conversation helpers
# ═══════════════════════════════════════════════════════════════════════════════

def chat(app, thread_id: str, user_input: str) -> str:
    """Send one user message; advance scripted LLM turn; return reply."""
    _SCRIPTED_LLM.next_turn()

    config = {"configurable": {"thread_id": thread_id}}
    inp    = {"messages": [HumanMessage(content=user_input)]}

    print(f"\n{'─'*62}")
    print(f"   {user_input}")
    print(f"{'─'*62}")

    result = app.invoke(inp, config=config)
    final  = result["messages"][-1]
    reply  = final.content if isinstance(final, AIMessage) else str(final)
    print(f"   {reply}")
    return reply


def show_history(app, thread_id: str, label: str = "") -> None:
    """Pretty-print the full checkpoint message history."""
    config = {"configurable": {"thread_id": thread_id}}
    state  = app.get_state(config)
    msgs   = state.values.get("messages", [])
    tag    = f"  [{label}]" if label else ""
    print(f"\n{'═'*62}")
    print(f"  CHECKPOINT HISTORY – thread '{thread_id}'{tag}")
    print(f"  {len(msgs)} messages stored")
    print(f"{'═'*62}")
    for i, m in enumerate(msgs):
        role    = type(m).__name__.replace("Message", "").upper()
        snippet = (m.content or "")[:90].replace("\n", " ")
        tc_tag  = ""
        if isinstance(m, AIMessage) and getattr(m, "tool_calls", None):
            tc_tag = f"  → [{', '.join(x['name'] for x in m.tool_calls)}]"
        print(f"  [{i:02d}] {role:<8} {snippet}{tc_tag}")
    print()


def recover_and_continue(checkpointer, thread_id: str, user_input: str) -> str:
    """
    Simulate a crash: build a brand-new graph instance using the EXISTING
    checkpointer, confirm all history is restored, then continue the
    conversation.
    """
    print(f"\n{'*'*62}")
    print(f"    SIMULATING CRASH — rebuilding graph from checkpoint …")
    app_new = build_graph(checkpointer)

    config  = {"configurable": {"thread_id": thread_id}}
    state   = app_new.get_state(config)
    n       = len(state.values.get("messages", []))
    print(f"    RECOVERED: {n} messages restored from MemorySaver")
    print(f"{'*'*62}")

    return chat(app_new, thread_id, user_input), app_new


# ═══════════════════════════════════════════════════════════════════════════════
# PART 6 ─ Full demo
# ═══════════════════════════════════════════════════════════════════════════════

def run_demo():
    checkpointer = MemorySaver()
    app          = build_graph(checkpointer)
    TID          = "portfolio-demo"

    print("═"*62)
    print("  Task 5 – LangGraph Persistent Conversation Agent")
    print(f"  Thread: {TID}   |   Checkpointer: MemorySaver")
    print("═"*62)

    # ── Segment 1: basics ─────────────────────────────────────────────────────
    print("\n SEGMENT 1 – Greeting & single-tool weather lookup")
    chat(app, TID, "Hello! What can you help me with?")
    chat(app, TID, "What's the weather in San Francisco?")

    # ── Segment 2: letter counting ────────────────────────────────────────────
    print("\n SEGMENT 2 – Letter counting (single, then parallel)")
    chat(app, TID, "How many s's are in 'Mississippi riverboats'?")
    chat(app, TID, "Are there more i's than s's in 'Mississippi riverboats'?")

    # ── Segment 3: sequential chain ───────────────────────────────────────────
    print("\n SEGMENT 3 – Sequential chain: count_letter × 2 → calculator")
    chat(app, TID,
         "What is the sin of the difference between the number of i's "
         "and s's in 'Mississippi riverboats'?")

    # ── Checkpoint snapshot before crash ─────────────────────────────────────
    show_history(app, TID, label="before crash")

    # ── Crash + Recovery ──────────────────────────────────────────────────────
    print("\n SEGMENT 4 – Context recall + CRASH / RECOVERY")
    _, reply = "dummy", chat(app, TID,
        "Which city's weather have we looked up so far in this conversation?")

    # Now simulate a crash and rebuild from checkpoint
    _, app = recover_and_continue(
        checkpointer, TID,
        "What's the weather in Tokyo? Count the 'o' letters in "
        "'Tokyo is a wonderful metropolis', get word stats for it, "
        "and compute the circle area using the letter count as the radius."
    )

    # ── Segment 5: deep sequential chain ─────────────────────────────────────
    print("\n SEGMENT 5 – Deep 4-step sequential chain (all 4 tools)")
    chat(app, TID,
         "Count 'e' and 'r' in 'The elephant remembered everything effortlessly'. "
         "Compute the hypotenuse of the right triangle whose legs are those counts. "
         "Then compute the circle area with that hypotenuse as the radius. "
         "Finally give me word stats for the sentence.")

    # ── Final checkpoint dump ─────────────────────────────────────────────────
    show_history(app, TID, label="final — full conversation")

    print("[DEMO COMPLETE]")


if __name__ == "__main__":
    run_demo()